In [156]:
import cobra
import cobra.test
import numpy as np
import sammi
import pandas as pd
from itertools import chain
from optlang.symbolics import Zero
from cobra.sampling import OptGPSampler

In [157]:
# Read the LB model
model_LB = cobra.io.read_sbml_model('Staphylococcus_epidermidis_ATCC_12228.xml')
model_LB.solver = 'glpk'

In [158]:
# Sort out LB growth medium
df = pd.read_excel('CarveMeMedia.ods')
df['agora'] = df['exchange'].apply(lambda x: x.removesuffix('_e') + '(e)')
df['agora'].replace('__', '_', regex=True, inplace=True)
df['keep'] = df['agora'].apply(lambda x: x in model_LB.exchanges)
df.set_index('agora', inplace=True)
growth_medium_LB = df[(df['id'] == 'LB') & df['keep']]['limit']

In [159]:
# Run the LB model with the defined growth medium
model_LB.medium = growth_medium_LB

In [160]:
# Read the M9 model
model_M9 = cobra.io.read_sbml_model('Staphylococcus_epidermidis_ATCC_12228.xml')
model_M9.solver = 'glpk'

In [161]:
# Sort out M9 growth medium
df = pd.read_excel('CarveMeMedia.ods')
df['agora'] = df['exchange'].apply(lambda x: x.removesuffix('_e') + '(e)')
df['agora'].replace('__', '_', regex=True, inplace=True)
df['keep'] = df['agora'].apply(lambda x: x in model_M9.exchanges)
df.set_index('agora', inplace=True)
growth_medium_M9 = df[(df['id'] == 'M9') & df['keep']]['limit']

In [162]:
# Run the M9 model with the defined growth medium
model_M9.medium = growth_medium_M9

In [163]:
# Solve the models
solution_LB = model_LB.optimize()
solution_M9 = model_M9.optimize()

In [164]:
solution_LB

,fluxes,reduced_costs
12DGR180ti,0.000000,0.000000e+00
1H2NPTH,0.000000,-2.293953e-18
1P4H2CBXLAH,0.000000,0.000000e+00
23DHMPO,0.000000,-5.568214e-18
23PDE2,0.000000,-1.734723e-18
...,...,...
r2137,0.401261,0.000000e+00
rtranscription,50.538622,0.000000e+00
sink_PGPm1[c],-0.000010,0.000000e+00
sink_gthrd(c),-0.401261,0.000000e+00


In [165]:
solution_M9

,fluxes,reduced_costs
12DGR180ti,0.0,0.000000e+00
1H2NPTH,0.0,3.153687e-15
1P4H2CBXLAH,0.0,0.000000e+00
23DHMPO,0.0,-2.110109e-31
23PDE2,0.0,0.000000e+00
...,...,...
r2137,0.0,0.000000e+00
rtranscription,0.0,0.000000e+00
sink_PGPm1[c],0.0,0.000000e+00
sink_gthrd(c),0.0,0.000000e+00


In [166]:
# Set a threshold for significant difference
threshold = 900

In [167]:
# Identify reactions with significant differences
different_reactions = set()
for reaction in model_LB.reactions:
    flux_LB = solution_LB.fluxes[reaction.id]
    flux_M9 = solution_M9.fluxes[reaction.id]
    if abs(flux_LB - flux_M9) > threshold:
        different_reactions.add(reaction.id)

In [168]:
# Print the different reactions
print("Different Reactions:")
for reaction_id in different_reactions:
    print(reaction_id)

Different Reactions:
EX_cys_L(e)
GLUt2r
PUNP1
EX_ac(e)
PGM
CYSDS
EX_dad_2(e)
r0570
PGK
PPM
EX_etoh(e)
NH4tb
ALAD_R
ACt2r
ACKr
ASPTA
EX_nh4(e)
HSDy
ATPS4
ETOHt
EX_glu_L(e)
THRt2r
CYSabc
PUNP2
H2Ot
EX_h(e)
ALCD2x
GAPD
MDH
ADNCNT3tc
PTAr
EX_adn(e)
EX_h2o(e)
G3PD2
FUM
DRPAr
DALAt2r
EX_thr_L(e)
FORt2r
ENO
EX_for(e)
DADNt2r
EX_ala_D(e)


In [169]:
diff_reac_list = list(different_reactions)

In [170]:
dat = [sammi.parser('Different reactions', diff_reac_list)]
sammi.plot(model_LB,dat)

C:\Users\sagni\anaconda3\lib\site-packages\cobra\core\group.py:107: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [151]:
# Identify reactions with significant differences
increased_flux_reactions = set()
decreased_flux_reactions = set()

for reaction in model_LB.reactions:
    flux_LB = solution_LB.fluxes[reaction.id]
    flux_M9 = solution_M9.fluxes[reaction.id]
    
    flux_difference = flux_LB - flux_M9
    
    if abs(flux_difference) > threshold:
        if flux_difference > 0:
            increased_flux_reactions.add(reaction.id)
        elif flux_difference < 0:
            decreased_flux_reactions.add(reaction.id)

In [152]:
# Print the reactions with increased flux
print("Reactions with Increased Flux in LB compared to M9:")
for reaction_id in increased_flux_reactions:
    print(reaction_id)

Reactions with Increased Flux in LB compared to M9:
EX_ac(e)
CYSDS
r0570
EX_etoh(e)
ASPTA
EX_nh4(e)
ATPS4
EX_glu_L(e)
THRt2r
CYSabc
PUNP2
GAPD
PTAr
EX_adn(e)
EX_h2o(e)
DRPAr
ENO
EX_for(e)
DADNt2r
EX_ala_D(e)


In [153]:
# Print the reactions with decreased flux
print("\nReactions with Decreased Flux in LB compared to M9:")
for reaction_id in decreased_flux_reactions:
    print(reaction_id)


Reactions with Decreased Flux in LB compared to M9:
EX_cys_L(e)
GLUt2r
PGM
EX_dad_2(e)
PGK
PPM
NH4tb
ALAD_R
ACt2r
ACKr
HSDy
ETOHt
H2Ot
EX_h(e)
ALCD2x
MDH
ADNCNT3tc
G3PD2
FUM
DALAt2r
EX_thr_L(e)
FORt2r
PUNP1


In [135]:
increased_flux_reactions_list = list(increased_flux_reactions)

In [136]:
dat_i = [sammi.parser('Increased flux in LB vs M9', increased_flux_reactions_list)]
sammi.plot(model_LB,dat_i)


In [154]:
decreased_flux_reactions_list = list(decreased_flux_reactions)

In [155]:
dat_d = [sammi.parser('Decreased flux in LB vs M9', decreased_flux_reactions_list)]
sammi.plot(model_LB,dat_d)